<a href="https://colab.research.google.com/github/SJinLee/cryptology/blob/main/05_1.AES_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.cs.uaf.edu/2015/spring/cs463/lecture/03_23_AES.html

In [3]:
def to_binary_list(m,ndigits=8):
    b = []
    for i in range(ndigits):
        b = [m%2] + b
        m = m//2
    return b
def to_num(b):
    m = 0
    for n in b:
        m = m*2+n
    return m

In [4]:
def add_gf256(a,b):
    return a^b
def mult_gf256(a,b):
    irr = 0x11b
    high = 0x100
    c = 0
    for bit in range(7,-1,-1):
        if b&(1<<bit):
            c=c^(a<<bit)

    for bit in range(7,-1,-1):
        if c & (high<<bit):
            c=c^(irr<<bit)
    return c

In [5]:
a = 0x57
b = 0x83
c = add_gf256(a,b)
hex(c)

'0xd4'

In [6]:
a = 0x57
b = 0x83
c = mult_gf256(a,b)
hex(c)

'0xc1'

In [7]:
to_binary_list(c)

[1, 1, 0, 0, 0, 0, 0, 1]

In [8]:
def gmix_column(r):
    a = [0]*4;
    b = [0]*4;
    # The array 'a' is simply a copy of the input array 'r'
    # The array 'b' is each element of the array 'a' multiplied by 2
    # in Rijndael's Galois field
    # a[n] ^ b[n] is element n multiplied by 3 in Rijndael's Galois field
    for c in range(4):
        a[c] = r[c]
        h = r[c] >> 7
        b[c] = (r[c] << 1) & 0x0ff
        b[c] ^= h * 0x1B # Rijndael's Galois field
    r2 = [0]*4
    r2[0] = b[0] ^ a[3] ^ a[2] ^ b[1] ^ a[1] # 2 * a0 + a3 + a2 + 3 * a1
    r2[1] = b[1] ^ a[0] ^ a[3] ^ b[2] ^ a[2] # 2 * a1 + a0 + a3 + 3 * a2
    r2[2] = b[2] ^ a[1] ^ a[0] ^ b[3] ^ a[3] # 2 * a2 + a1 + a0 + 3 * a3
    r2[3] = b[3] ^ a[2] ^ a[1] ^ b[0] ^ a[0] # 2 * a3 + a2 + a1 + 3 * a0
    return r2

In [9]:
c = [0x01,0x02,0x03,0x04]
gmix_column(c)

[3, 4, 9, 10]

In [10]:
def calc_all_inv():
    inv = []
    for a in range(256):
        for b in range(256):
            c = mult_gf256(a,b)
            if c==1:
                break
        if c==1:
            inv.append(b)
        else:
            inv.append(0)
    return inv

In [11]:
inv = calc_all_inv()
[' '.join([x.to_bytes(1,'big').hex() for x in inv[i:i+16]]) for i in range(0,256,16)]

['00 01 8d f6 cb 52 7b d1 e8 4f 29 c0 b0 e1 e5 c7',
 '74 b4 aa 4b 99 2b 60 5f 58 3f fd cc ff 40 ee b2',
 '3a 6e 5a f1 55 4d a8 c9 c1 0a 98 15 30 44 a2 c2',
 '2c 45 92 6c f3 39 66 42 f2 35 20 6f 77 bb 59 19',
 '1d fe 37 67 2d 31 f5 69 a7 64 ab 13 54 25 e9 09',
 'ed 5c 05 ca 4c 24 87 bf 18 3e 22 f0 51 ec 61 17',
 '16 5e af d3 49 a6 36 43 f4 47 91 df 33 93 21 3b',
 '79 b7 97 85 10 b5 ba 3c b6 70 d0 06 a1 fa 81 82',
 '83 7e 7f 80 96 73 be 56 9b 9e 95 d9 f7 02 b9 a4',
 'de 6a 32 6d d8 8a 84 72 2a 14 9f 88 f9 dc 89 9a',
 'fb 7c 2e c3 8f b8 65 48 26 c8 12 4a ce e7 d2 62',
 '0c e0 1f ef 11 75 78 71 a5 8e 76 3d bd bc 86 57',
 '0b 28 2f a3 da d4 e4 0f a9 27 53 04 1b fc ac e6',
 '7a 07 ae 63 c5 db e2 ea 94 8b c4 d5 9d f8 90 6b',
 'b1 0d d6 eb c6 0e cf ad 08 4e d7 e3 5d 50 1e b3',
 '5b 23 38 34 68 46 03 8c dd 9c 7d a0 cd 1a 41 1c']

In [12]:
def affine(b):
    mat = [[1,0,0,0,1,1,1,1],
           [1,1,0,0,0,1,1,1],
           [1,1,1,0,0,0,1,1],
           [1,1,1,1,0,0,0,1],
           [1,1,1,1,1,0,0,0],
           [0,1,1,1,1,1,0,0],
           [0,0,1,1,1,1,1,0],
           [0,0,0,1,1,1,1,1]]
    const = [1,1,0,0,0,1,1,0]
    bb = to_binary_list(b)[::-1]
    cc = []
    for i in range(8):
        c = sum([x*y for x,y in zip(mat[i],bb)]) + const[i]
        cc = cc + [c % 2]
    return to_num(cc[::-1])

In [13]:
sbox = {i:affine(x) for i,x in enumerate(inv)}

In [14]:
[' '.join([sbox[x].to_bytes(1,'big').hex() for x in range(i,i+16)]) for i in range(0,256,16)]

['63 7c 77 7b f2 6b 6f c5 30 01 67 2b fe d7 ab 76',
 'ca 82 c9 7d fa 59 47 f0 ad d4 a2 af 9c a4 72 c0',
 'b7 fd 93 26 36 3f f7 cc 34 a5 e5 f1 71 d8 31 15',
 '04 c7 23 c3 18 96 05 9a 07 12 80 e2 eb 27 b2 75',
 '09 83 2c 1a 1b 6e 5a a0 52 3b d6 b3 29 e3 2f 84',
 '53 d1 00 ed 20 fc b1 5b 6a cb be 39 4a 4c 58 cf',
 'd0 ef aa fb 43 4d 33 85 45 f9 02 7f 50 3c 9f a8',
 '51 a3 40 8f 92 9d 38 f5 bc b6 da 21 10 ff f3 d2',
 'cd 0c 13 ec 5f 97 44 17 c4 a7 7e 3d 64 5d 19 73',
 '60 81 4f dc 22 2a 90 88 46 ee b8 14 de 5e 0b db',
 'e0 32 3a 0a 49 06 24 5c c2 d3 ac 62 91 95 e4 79',
 'e7 c8 37 6d 8d d5 4e a9 6c 56 f4 ea 65 7a ae 08',
 'ba 78 25 2e 1c a6 b4 c6 e8 dd 74 1f 4b bd 8b 8a',
 '70 3e b5 66 48 03 f6 0e 61 35 57 b9 86 c1 1d 9e',
 'e1 f8 98 11 69 d9 8e 94 9b 1e 87 e9 ce 55 28 df',
 '8c a1 89 0d bf e6 42 68 41 99 2d 0f b0 54 bb 16']

In [15]:
def RotWord(w,shift=1):
    if shift==0:
        return w[:]
    return w[shift:] + w[:shift]

In [16]:
def SubByte(b):
    return sbox[b]
def SubWord(w):
    return [SubByte(x) for x in w]

In [17]:
def SubBytes(bytes):
    return [sbox[b] for b in bytes]
def ShiftRows(words):
    ret = []
    for s in range(4):
        ret += RotWord(words[s*4:s*4+4],shift=s)
    return ret
def MixColumns(words):
    for i in range(4):
        r = [words[4*k+i] for k in range(4)]
        r = gmix_column(r)
        for j in range(4):
            words[4*j+i] = r[j]

In [18]:
def get_rcon(Nr):
    rcon = [0,1,2]
    for i in range(3,Nr+1):
        x = mult_gf256(rcon[i-1],2)
        rcon.append(x)
    return rcon

In [19]:
rcon = get_rcon(14)
' '.join([x.to_bytes(1,'big').hex() for x in rcon])

'00 01 02 04 08 10 20 40 80 1b 36 6c d8 ab 4d'

In [20]:
def key_expansion(key,nk,nr,nb):
    rcon = [[x,0,0,0] for x in get_rcon(nr)]
    w = []
    for i in range(nk):
        w.append(key[i*4:i*4+4])
    for i in range(nk,nb*(nr+1)):
        temp = w[i-1]
        if i%nk==0:
            temp = [x^y for x,y in zip(SubWord(RotWord(temp)),rcon[i//nk])]
        elif nk>6 and i%nk==4:
            temp = SubWord(temp)
        w.append([x^y for x,y in zip(w[i-nk],temp)])
    return w

In [21]:
nk2nr = {4:10,6:12,8:14}
nk = 4
nr = nk2nr[nk]
nb = 4
K = [i for i in range(nk*4)]
M = [i+16 for i in range(nk*4)]

In [22]:
' '.join([x.to_bytes(1,'big').hex() for x in K])

'00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f'

In [23]:
' '.join([x.to_bytes(1,'big').hex() for x in M])

'10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f'

In [24]:
w = key_expansion(K,nk,nr,nb)

In [39]:
mm = M
for i in range(0,nr):
    mm = [x^y for x,y in zip(mm,sum(w[4*i:4*i+4],[]))]
    print(i,'key',mm)
    mm = SubBytes(mm)
    print(i,'sbox',mm)
    mm = ShiftRows(mm)
    print(i,'shift',mm)
    if i<nr-1:
        MixColumns(mm)
        print(i,'mix',mm)
C = [x^y for x,y in zip(mm,sum(w[4*nr:4*nr+4],[]))]
C

0 key [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]
0 sbox [202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202]
0 shift [202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202]
0 mix [202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202, 202]
1 key [28, 96, 190, 55, 24, 101, 184, 48, 16, 108, 178, 59, 28, 97, 188, 52]
1 sbox [156, 208, 174, 154, 173, 77, 108, 4, 202, 80, 55, 226, 156, 239, 101, 24]
1 shift [156, 208, 174, 154, 77, 108, 4, 173, 55, 226, 202, 80, 24, 156, 239, 101]
1 mix [219, 113, 110, 246, 71, 169, 12, 78, 151, 220, 15, 56, 245, 198, 226, 130]
2 key [109, 227, 161, 253, 35, 148, 177, 191, 41, 71, 202, 56, 157, 246, 81, 124]
2 sbox [60, 17, 50, 84, 38, 34, 200, 8, 165, 160, 116, 7, 94, 66, 209, 16]
2 shift [60, 17, 50, 84, 34, 200, 8, 38, 116, 7, 165, 160, 16, 94, 66, 209]
2 mix [122, 56, 155, 179, 244, 205, 148, 50, 198, 53, 173, 65, 50, 64, 127, 195]
3 key [204, 199, 239, 253, 

[193, 52, 224, 212, 50, 193, 211, 86, 46, 81, 208, 137, 100, 104, 12, 36]

In [26]:
def invaffine(b):
    mati = [[0,0,1,0,0,1,0,1],
           [1,0,0,1,0,0,1,0],
           [0,1,0,0,1,0,0,1],
           [1,0,1,0,0,1,0,0],
           [0,1,0,1,0,0,1,0],
           [0,0,1,0,1,0,0,1],
           [1,0,0,1,0,1,0,0],
           [0,1,0,0,1,0,1,0]]
    const = [1,1,0,0,0,1,1,0]
    bpc = [x+y for x,y in zip(to_binary_list(b)[::-1], const)]
    cc = []
    for i in range(8):
        c = sum([x*y for x,y in zip(mati[i],bpc)])
        cc = cc + [c % 2]
    return to_num(cc[::-1])

In [46]:
invsbox2 = {x:inv[invaffine(x)] for x in range(256)}

In [47]:
[' '.join([invsbox2[x].to_bytes(1,'big').hex() for x in range(i,i+16)]) for i in range(0,256,16)]

['52 09 6a d5 30 36 a5 38 bf 40 a3 9e 81 f3 d7 fb',
 '7c e3 39 82 9b 2f ff 87 34 8e 43 44 c4 de e9 cb',
 '54 7b 94 32 a6 c2 23 3d ee 4c 95 0b 42 fa c3 4e',
 '08 2e a1 66 28 d9 24 b2 76 5b a2 49 6d 8b d1 25',
 '72 f8 f6 64 86 68 98 16 d4 a4 5c cc 5d 65 b6 92',
 '6c 70 48 50 fd ed b9 da 5e 15 46 57 a7 8d 9d 84',
 '90 d8 ab 00 8c bc d3 0a f7 e4 58 05 b8 b3 45 06',
 'd0 2c 1e 8f ca 3f 0f 02 c1 af bd 03 01 13 8a 6b',
 '3a 91 11 41 4f 67 dc ea 97 f2 cf ce f0 b4 e6 73',
 '96 ac 74 22 e7 ad 35 85 e2 f9 37 e8 1c 75 df 6e',
 '47 f1 1a 71 1d 29 c5 89 6f b7 62 0e aa 18 be 1b',
 'fc 56 3e 4b c6 d2 79 20 9a db c0 fe 78 cd 5a f4',
 '1f dd a8 33 88 07 c7 31 b1 12 10 59 27 80 ec 5f',
 '60 51 7f a9 19 b5 4a 0d 2d e5 7a 9f 93 c9 9c ef',
 'a0 e0 3b 4d ae 2a f5 b0 c8 eb bb 3c 83 53 99 61',
 '17 2b 04 7e ba 77 d6 26 e1 69 14 63 55 21 0c 7d']

In [27]:
invsbox = {sbox[i]:i for i in sbox}

In [28]:
[' '.join([invsbox[x].to_bytes(1,'big').hex() for x in range(i,i+16)]) for i in range(0,256,16)]

['52 09 6a d5 30 36 a5 38 bf 40 a3 9e 81 f3 d7 fb',
 '7c e3 39 82 9b 2f ff 87 34 8e 43 44 c4 de e9 cb',
 '54 7b 94 32 a6 c2 23 3d ee 4c 95 0b 42 fa c3 4e',
 '08 2e a1 66 28 d9 24 b2 76 5b a2 49 6d 8b d1 25',
 '72 f8 f6 64 86 68 98 16 d4 a4 5c cc 5d 65 b6 92',
 '6c 70 48 50 fd ed b9 da 5e 15 46 57 a7 8d 9d 84',
 '90 d8 ab 00 8c bc d3 0a f7 e4 58 05 b8 b3 45 06',
 'd0 2c 1e 8f ca 3f 0f 02 c1 af bd 03 01 13 8a 6b',
 '3a 91 11 41 4f 67 dc ea 97 f2 cf ce f0 b4 e6 73',
 '96 ac 74 22 e7 ad 35 85 e2 f9 37 e8 1c 75 df 6e',
 '47 f1 1a 71 1d 29 c5 89 6f b7 62 0e aa 18 be 1b',
 'fc 56 3e 4b c6 d2 79 20 9a db c0 fe 78 cd 5a f4',
 '1f dd a8 33 88 07 c7 31 b1 12 10 59 27 80 ec 5f',
 '60 51 7f a9 19 b5 4a 0d 2d e5 7a 9f 93 c9 9c ef',
 'a0 e0 3b 4d ae 2a f5 b0 c8 eb bb 3c 83 53 99 61',
 '17 2b 04 7e ba 77 d6 26 e1 69 14 63 55 21 0c 7d']

In [41]:
def InvRotWord(w,shift=1):
    if shift==0:
        return w[:]
    return w[-shift:] + w[:-shift]
def InvSubBytes(bytes):
    return [invsbox[b] for b in bytes]
def InvShiftRows(words):
    ret = []
    for s in range(4):
        ret += InvRotWord(words[s*4:s*4+4],shift=s)
    return ret
def inv_gmix_column(r):
    a = [0]*4;
    b = [0]*4;
    c = [0]*4;
    d = [0]*4;
    # The array 'a' is simply a copy of the input array 'r'
    # The array 'b' is each element of the array 'a' multiplied by 2
    # The array 'c' is each element of the array 'a' multiplied by 4
    # The array 'd' is each element of the array 'a' multiplied by 8
    # in Rijndael's Galois field
    # a[n] ^ b[n] is element n multiplied by 3 in Rijndael's Galois field
    for i in range(4):
        a[i] = r[i]
        h = r[i] >> 7
        b[i] = (r[i] << 1) & 0x0ff
        b[i] ^= h * 0x1B # Rijndael's Galois field
        h = b[i] >> 7
        c[i] = (b[i] << 1) & 0x0ff
        c[i] ^= h * 0x1B # Rijndael's Galois field
        h = c[i] >> 7
        d[i] = (c[i] << 1) & 0x0ff
        d[i] ^= h * 0x1B # Rijndael's Galois field
    r2 = [0]*4
    r2[0] = d[0] ^ c[0] ^ b[0] ^ d[3] ^ a[3] ^ d[2] ^ c[2] ^ a[2] ^ d[1] ^ b[1] ^ a[1] # 14 * a0 + 9 * a3 + 13 * a2 + 11 * a1
    r2[1] = d[1] ^ c[1] ^ b[1] ^ d[0] ^ a[0] ^ d[3] ^ c[3] ^ a[3] ^ d[2] ^ b[2] ^ a[2] # 14 * a1 + 9 * a0 + 13 * a3 + 11 * a2
    r2[2] = d[2] ^ c[2] ^ b[2] ^ d[1] ^ a[1] ^ d[0] ^ c[0] ^ a[0] ^ d[3] ^ b[3] ^ a[3] # 14 * a2 + 9 * a1 + 13 * a0 + 11 * a3
    r2[3] = d[3] ^ c[3] ^ b[3] ^ d[2] ^ a[2] ^ d[1] ^ c[1] ^ a[1] ^ d[0] ^ b[0] ^ a[0] # 14 * a3 + 9 * a2 + 13 * a1 + 11 * a0
    return r2
def InvMixColumns(words):
    for i in range(4):
        r = [words[4*k+i] for k in range(4)]
        r = inv_gmix_column(r)
        for j in range(4):
            words[4*j+i] = r[j]

In [44]:
cc = C
for i in range(nr,0,-1):
    cc = [x^y for x,y in zip(cc,sum(w[4*i:4*i+4],[]))]
    print(i,'key',cc)
    if i<nr:
        InvMixColumns(cc)
        print(i,'invmix',cc)
    cc = InvShiftRows(cc)
    print(i,'invshift',cc)
    cc = InvSubBytes(cc)
    print(i,'invsbox',cc)
mm = [x^y for x,y in zip(cc,sum(w[:4],[]))]
mm

10 key [210, 37, 253, 171, 209, 85, 153, 65, 221, 86, 119, 2, 41, 67, 60, 225]
10 invshift [210, 37, 253, 171, 65, 209, 85, 153, 119, 2, 221, 86, 67, 60, 225, 41]
10 invsbox [127, 194, 33, 14, 248, 81, 237, 249, 2, 106, 201, 185, 100, 109, 224, 76]
9 key [43, 91, 19, 223, 8, 212, 186, 145, 18, 249, 36, 37, 218, 65, 119, 2]
9 invmix [123, 116, 237, 138, 171, 120, 239, 148, 143, 184, 53, 162, 180, 131, 205, 213]
9 invshift [123, 116, 237, 138, 148, 171, 120, 239, 53, 162, 143, 184, 131, 205, 213, 180]
9 invsbox [3, 202, 83, 207, 231, 14, 193, 97, 217, 26, 115, 154, 65, 128, 181, 198]
8 key [68, 137, 212, 250, 67, 18, 164, 216, 57, 12, 201, 110, 239, 63, 207, 20]
8 invmix [166, 25, 171, 118, 50, 5, 34, 28, 98, 215, 90, 82, 39, 99, 165, 96]
8 invshift [166, 25, 171, 118, 28, 50, 5, 34, 90, 82, 98, 215, 99, 165, 96, 39]
8 invsbox [197, 142, 14, 15, 196, 161, 54, 148, 70, 72, 171, 13, 0, 41, 144, 61]
7 key [209, 119, 126, 21, 39, 254, 212, 24, 2, 66, 116, 64, 78, 128, 80, 27]
7 invmix [90, 1

[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

In [43]:
M

[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]